In [9]:
import pythtb as tb

In [10]:
import numpy as np
from numpy import linalg as la

In [72]:
import tenpy.linalg.np_conserved as npc
from tenpy.networks.site import SpinHalfFermionSite
from tenpy.networks.mps import MPS
from tenpy.networks.mpo import MPO
from tenpy.linalg.np_conserved import Array
from tenpy.algorithms.truncation import svd_theta

# Parton Model Parameters

First of all, we need to set up the tight-binding model correctly.

## Model/Hamiltonian Parameters

In [63]:
# set and solve the parton band model
t, td = 1., 0.5j
Lx ,Ly =6 , 4
L = Lx * Ly * 2
# parameters for MPS
chimax = 800
cutoff = 1e-10

## Geometry/Lattice Parameter

In [64]:
# define lattice vectors
lat=[[2.0,0.0],[0.0,1.0]]
# define coordinates of orbitals
orb=[[0.0, 0.0], [0.5, 0.0]]
# make two dimensional tight-binding model
parton=tb.tb_model(2,2,lat,orb)
# set model parameters
# add first neighbour hoppings
parton.set_hop(t, 0, 1, [0, 0])
parton.set_hop(t, 1, 0, [1, 0])
parton.set_hop(t, 0, 0, [0, 1])
parton.set_hop(-t, 1, 1, [0, 1])
# add second neighbour hoppings
parton.set_hop(td, 0, 1, [0, 1])
parton.set_hop(td, 1, 0, [0, 1])
parton.set_hop(-td, 1, 0, [1, 1])
parton.set_hop(-td, 0, 1, [-1,1])

# PieceCutting

In [65]:
#We first work with a finite system
# cutout finite model first along direction x without PBC
tmp_parton=parton.cut_piece(Lx,0,glue_edgs=True)
# cutout also along y direction with PBC
fin_parton=tmp_parton.cut_piece(Ly,1,glue_edgs=True)
# solve finite model
(evals,evecs) =fin_parton.solve_all(eig_vectors=True)
# computes hybrid Wannier centers (and functions) for the filled band. This procedure reduces truncation errors.
# We work with Wannier basis to reduce long range entanglement and truncation error 
filled=evecs[0:Lx*Ly,:]
pos_matx=fin_parton.position_matrix(evecs[:Lx*Ly],0)
pos_maty=fin_parton.position_matrix(evecs[:Lx*Ly],1)
pos_mat=Ly*pos_maty+pos_matx
Wanniercenter, Wannier= LA.eig(pos_mat)
parton_hwf=np.matmul(Wannier.transpose() ,filled)

# MPS Representation

Secondly, we need to use the given wavefunction to create the corresponding (i)MPS representation.

Reference $0$: `arxiv 1910.11011`

In [66]:
#1. Represent fermion vacuum as MPS: |0>=|0000...>
site = SpinHalfFermionSite()
psi = MPS.from_product_state([site]*L, ['empty']*L, 'finite')

In [67]:
# show the corresponding indices of all the labels
site.state_labels

{'down': 0, 'empty': 1, 'full': 2, 'up': 3}

In [68]:
# Aml is the corresponding matrix in reference 0
Aml = parton_hwf

chinfo, p_leg = site.leg.chinfo, site.leg
print(f'chinfo = {chinfo}')
print(f'p_leg = {p_leg}')

chinfo = ChargeInfo([1, 1], ['N', 'Sz'])
p_leg =  +1
0 [[ 1 -1] 
1  [ 0  0] 
2  [ 2  0] 
3  [ 1  1]]
4          


In [69]:
def Ws(
    Aml, 
    m, 
    L, # just automatically set it...
    spin='up', 
    bc='infinite',
):
    """
        Given a `Aml` and the corresponding `m`, 
        create and return the wanted MPO Wm
    """
    site = SpinHalfFermionSite()
    if spin not in ['up', 'down']:
        raise ValueError('Not such spin! UP or DOWN, choose one, thanks!')
    assert bc in ['finite', 'infinite', 'segment']
    
    op = 'Cdu'
    if spin=='down':
        op = 'Cdd'
    "MPO for d_m^\dag (spin up)"
    Cdu, Id = [site.get_op(op) for op in ('Cdu', 'Id')]
    Wm = []
    wL_leg = npc.LegCharge.from_qflat(chinfo,  [op.qtotal for op in [Cdu, Id]] , qconj=+1) # set charge on the leg
    for l in range(L):
        Wml_grid = [[Id, None],
                    [Aml[m,l]*Cdu, Id]]
        Wml = npc.grid_outer(Wml_grid, [wL_leg, wL_leg.conj()],grid_labels=['wL', 'wR']) # wL, wR, p, p*
        Wm.append(Wml)
#         print(Wml)
    if bc=='finite': # Boundary case, see the Eq.2 of ref 0
        Wm[0] = Wm[0][1:2,:] # second row
        Wm[-1] = Wm[-1][:, 0:1] # first column
    dm = MPO(sites = [site]*L, Ws = Wm, bc=bc)
    return dm

def generate_mps(Aml, spin='up', bc='infinite'):
    
    #1. Represent fermion vacuum as MPS: |0>=|0000...>
    site = SpinHalfFermionSite()
    psi = MPS.from_product_state([site]*L, ['empty']*L, bc)
    chinfo, p_leg = site.leg.chinfo, site.leg
    
    #4. Apply MPO to MPS and then perform SVD compressions: d_m^dag |0>
    for m in range(L//2): # v=1
        dm = Ws(Aml, m, L, spin=spin, bc=bc)
        print(dm)
        dm.apply(
            psi, 
            dict(
                chi_max=chimax, 
                svd_min=cutoff,
                compression_method='SVD',
            ),
        )
    return psi
#         for i in range(L):
#             B = psi.get_B(i, form='B')
#             Wml = Wm[i]
#             B = npc.tensordot(Wml, B, ['p*', 'p']) # wL wR p vL vR 
#             B.itranspose(['vL', 'wL', 'p', 'vR', 'wR'])
#             B = B.combine_legs([('vL', 'wL'), ('vR', 'wR')], qconj=[1, -1])
#             B.iset_leg_labels(['vL', 'p', 'vR'])
#             psi.set_B(i, B, form=None)
#         psi.canonical_form() # to BBB...

#         psi.compress_svd(
#             trunc_par=dict(
#                 chi_max=chimax,
#                 svd_min=cutoff,
#             )
#         )

#     # SVD compression
#     for i in range(L-1):
#         theta = psi.get_theta(i, n=2)
#         theta = theta.combine_legs([('vL', 'p0'), ('p1', 'vR')], qconj=[1, -1]) # vL.p0 p1.vR
#         A0, S1, B1, error, renorm = svd_theta(theta, {'svd_min': cutoff,
#                                                       'chi_max': chimax})
#         psi.set_B(i, A0.split_legs().replace_label('p0', 'p'), form='A')
#         psi.set_SR(i, S1)
#         psi.set_B(i+1, B1.split_legs().replace_label('p1', 'p'), form='B')
        

In [71]:
spin = 'up'
psi_up = generate_mps(Aml, spin=spin, bc='infinite')

spin = 'down'
psi_down = generate_mps(Aml, spin=spin, bc='infinite')

Reading 'compression_method'='SVD' for config MPO_apply


/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/tools/params.py:225: UserWarning: unused options for config MPO_apply:
['chi_max', 'svd_min']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Reading 'compression_method'='SVD' for config MPO_apply


/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/tools/params.py:225: UserWarning: unused options for config MPO_apply:
['chi_max', 'svd_min']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Reading 'compression_method'='SVD' for config MPO_apply


...

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/tools/params.py:225: UserWarning: unused options for config MPO_apply:
['chi_max', 'svd_min']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Reading 'compression_method'='SVD' for config MPO_apply


/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/tools/params.py:225: UserWarning: unused options for config MPO_apply:
['chi_max', 'svd_min']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Reading 'compression_method'='SVD' for config MPO_apply


/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/tools/params.py:225: UserWarning: unused options for config MPO_apply:
['chi_max', 'svd_min']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


In [76]:
# Gutzwiller projection to physical Hilbert space
def get_physical_psi(psi_input):
    psi = psi_input.copy()
    chinfo, p_leg = site.leg.chinfo, site.leg
    GPOls = np.array([[1, 0, 0, 0],
                      [0, 0, 0, 0],
                      [0, 0, 0, 0],
                      [0, 0, 0, 1]])
    GPOl=Array.from_ndarray(GPOls, [p_leg, p_leg.conj()]) # p, p*
    GPOl.iset_leg_labels(['p', 'p*'])
    for i in range(psi.L):
        psi.apply_local_op(i, GPOl)
    psi.canonical_form()
    return psi

In [ ]:
psi_up_physical = get_physical_psi(psi_up)

/home/jerry/anaconda3/lib/python3.7/site-packages/tenpy/networks/mps.py:2433: UserWarning: canonical_form_infinite: project to smaller bond dimension
  self.canonical_form_infinite(renormalize)


In [ ]:
psi_up_physical.entanglement_spect

TODO:
 - Fix the computing for `bc='finite'`